# Neighborhoods in Toronto

## 1. Website scraping using BeautifulSoup package

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [7]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')

In [8]:
my_table = soup.find('table',{'class':'wikitable sortable'})
contents = my_table.findAll('td')

## 2. Transform the data into a pandas dataframe

In [9]:
len(contents)

864

In [10]:
row_num_total = int(len(contents)/3)

In [11]:
# get all table contents in a list
t = [r.get_text() for r in contents]

In [14]:
#convert the list to data frame of the correct format
df = pd.DataFrame(np.array(t).reshape(row_num_total,3), columns = list(['PostCode','Borough', 'Neighbourhood']))

In [15]:
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [17]:
# remove the '\n' at the end of each Neighbourhood
df['Neighbourhood'] = df['Neighbourhood'].str.replace(r'\n', '')

In [18]:
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 3. Ignore cells with a borough that is Not assigned

In [20]:
df1 = df[df.Borough != 'Not assigned']

In [21]:
df1.head()

,PostCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## 4. Combine neighborhoods existing in one postal code area

In [25]:
df2 = df1.groupby(['PostCode', 'Borough'], sort = False).agg(', '.join)
df2.reset_index(inplace = True)

In [26]:
df2.head()

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


## 5. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [27]:
df2['Neighbourhood'] = df2['Neighbourhood'].replace('Not assigned', df2['Borough'])

In [28]:
df2.head()

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


## 6. Print the number of rows

In [29]:
df2.shape

(103, 3)

### Woohoo, that's the end. 

## Oh, not yet.

## 7. Create a dataframe containing geographical coordinates of the neighborhoods

In [37]:
# read in geo information as data frame
geo_url = "https://cocl.us/Geospatial_data"
df_geo = pd.read_csv(geo_url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
# join these two data frames
df_merged = pd.merge(left=df2,right=df_geo , left_on='PostCode', right_on='Postal Code')

In [39]:
df_merged.head()

,PostCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494


In [42]:
df_merged.drop('Postal Code', axis=1, inplace = True)

In [43]:
df_merged.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
